# RNN

Given a sequence, predict the topic distribution for it

In [17]:
import pandas as pd
import numpy as np

# the dists are saved as comma separated values in a string (because pandas can't save datatype as int)
# so to read as numeric-values, you must split on spaces and convert to float.
# Michael says there is an efficient way to do this.

Here is our dataset

In [18]:
df = pd.read_csv('data.csv')
df.head()

,sentences,dists
0,At least 100 families receiving housing benefi...,-6.63052740918379 -7.265516111874736 -6.913763...
1,"More 30 families given staggering # 1,500 week...",-7.830457309882772 -7.835212219711822 -5.67915...
2,"Of 100 families , 60 rent paid state value # 5...",-7.872029012645258 -6.913763006152177 -5.67915...
3,At time millions people struggling get housing...,-6.63052740918379 -6.443461174803561 -7.839491...
4,"Luxury : The kind upmarket homes Kensington , ...",-6.899740172840747 -inf -4.813336632402851 -7....


In [19]:
# convert back to original distribution
df['dists'] = df['dists'].str.split(" ")
df['dists'] = df['dists'].apply(lambda x: [np.e ** float(i) for i in x])

In [20]:
np.sum(df['dists'][0])

0.02244788873638659

Let the ML begin

In [21]:
import unicodedata
import string
import re
import random
import time
import math
import csv

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="6,7"

In [22]:
USE_CUDA = False

Define our vocabulary class

In [23]:
SOS_token = 0
EOS_token = 1

class Voc:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2 # Count SOS and EOS
      
    def index_words(self, sentence):
        for word in sentence.split(' '):
            self.index_word(word)

    def index_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Methods for cleaning up data

In [24]:
# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

Lets actually use the data now

In [25]:
MAX_SENTENCES = 200

# subset of the data for training
training_df = df[:MAX_SENTENCES]

# choose a random sentence and its corresponding distribution (label)
def choose_random_training_pair():
    training_pair = training_df.loc[random.randint(0, training_df.shape[0])]

    sentence = training_pair['sentences']
    distribution = training_pair['dists']

    return sentence, distribution

def generate_vocabulary(sentences):
    # create the vocabulary
    vocabulary = Voc('source_identification')
    for sentence in sentences:
        vocabulary.index_words(sentence)

    return vocabulary

In [26]:
vocabulary = generate_vocabulary(training_df['sentences'].tolist())
n_words = vocabulary.n_words

Lets actually do some ML now.

Here are some sentences --> variable functions

In [27]:
# Return a list of indexes, one for each word in the sentence
def indexes_from_sentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')]

def variable_from_sentence(voc, sentence):
    indexes = indexes_from_sentence(voc, sentence)
    indexes.append(EOS_token)
    var = Variable(torch.LongTensor(indexes).view(-1, 1))
    if USE_CUDA: var = var.cuda()
    return var

def tensor_from_distribution(distribution):
    return Variable(torch.FloatTensor(distribution).view(-1, 1), requires_grad=True)

And now here's the RNN

In [28]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        # embedding contains input_size vectors of size hidden_size.
        # this performs the word embeddings
        self.embedding = nn.Embedding(input_size, hidden_size)

        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        
    def forward(self, word_inputs, hidden):
        # Note: we run this all at once (over the whole input sequence)
        seq_len = len(word_inputs)
        
        # embedding: (sentence length, batch size, word embedding length)
        embedded = self.embedding(word_inputs).view(seq_len, 1, -1)
        
        # GRU:
        #    input: (length of sequence, batch size, size of var in sequence)
        #    output: 
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        if USE_CUDA: hidden = hidden.cuda()
        return hidden

In [29]:
class classifierNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(classifierNN, self).__init__()
        
        self.input_size = input_size
        self.output_size = output_size
        
        self.i2o = nn.Linear(input_size, output_size)
    
    def forward(self, result_input):
        return self.i2o(result_input)

Here's our training function

In [31]:
teacher_forcing_ratio = 0.5
clip = 5.0

def train(input_var, target, rnn, optimizer, criterion):

    # Zero gradients of both optimizers
    optimizer.zero_grad()
    loss = 0 # Added onto for each word

    # Get size of input and target sentences
    # input_length = input_variable.size()[0]

    # Run sentence through rnn
    hidden_var = rnn.init_hidden()
    output_var, hidden_var = rnn(input_var, hidden_var)
    
    # compute loss
    loss = criterion(output_var[-1], target)

    # run backprop
    loss.backward()
    torch.nn.utils.clip_grad_norm(rnn.parameters(), clip)
    optimizer.step()
    
    return loss.data[0]

Helper functions for time

In [32]:
def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))

This is where we actually train the RNN  
.  
.  
.  
.  
.  
.  
.  
.  
.  
after we initialize stuff

In [33]:
hidden_size = 500 # size of each word embedding
n_layers = 2 # number of layers for the RNN
dropout_p = 0.05 # we never use this

# Initialize the RNN
rnn = RNN(n_words, hidden_size, n_layers)

num_topics = 24

# Initialize the classifier NN
classifier = classifierNN(hidden_size, num_topics)

# Move models to GPU
if USE_CUDA:
    rnn.cuda()

# Initialize optimizers and criterion
learning_rate = 0.0001
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

In [34]:
# Configuring training
n_epochs = 1000
plot_every = 20
print_every = 10

# Keep track of time elapsed and running averages
plot_losses = []
print_loss_total = 0 # Reset every print_every
plot_loss_total = 0 # Reset every plot_every

Aaaaand now we actually start training

In [35]:
start = time.time()

for epoch in range(1, n_epochs + 1):
    
    # Get training data for this cycle
    sentence, distribution = choose_random_training_pair()
    
    training_sentence = variable_from_sentence(vocabulary, sentence)
    target_distribution = tensor_from_distribution(distribution)

    # Run the train function
    loss = train(training_sentence, target_distribution, rnn, optimizer, criterion)

    # Keep track of loss
    print_loss_total += loss
    plot_loss_total += loss

    if epoch % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print_summary = '%s (%d %d%%) %.4f' % (time_since(start, epoch / n_epochs), epoch, epoch / n_epochs * 100, print_loss_avg)
        print(print_summary)

    if epoch % plot_every == 0:
        plot_loss_avg = plot_loss_total / plot_every
        plot_losses.append(plot_loss_avg)
        plot_loss_total = 0

Variable containing:

Columns 0 to 9 
 0.0718 -0.0444  0.0503  0.1155  0.2382 -0.1270 -0.2175 -0.0405 -0.0161 -0.1410

Columns 10 to 19 
-0.0534  0.2328  0.0237  0.0722 -0.0093  0.0970  0.0476  0.1252 -0.0963  0.1511

Columns 20 to 29 
-0.1360 -0.1065  0.0534  0.1789  0.1194  0.1297 -0.0641 -0.1509  0.0713 -0.0513

Columns 30 to 39 
 0.0520 -0.0236  0.1997 -0.0050 -0.0200 -0.1702 -0.0723 -0.0382 -0.0044 -0.0123

Columns 40 to 49 
 0.2125 -0.0207  0.1366  0.0581  0.1211 -0.0630 -0.1311 -0.0710 -0.0072 -0.0984

Columns 50 to 59 
-0.0053 -0.0205  0.0473  0.0987  0.0676 -0.1871 -0.2164 -0.1523 -0.0425  0.2094

Columns 60 to 69 
 0.1477 -0.1624  0.0255  0.1580 -0.1315  0.1256  0.1374  0.2881 -0.0272  0.0577

Columns 70 to 79 
-0.2502  0.2343 -0.0010  0.2240 -0.0500 -0.1467  0.1655 -0.0345  0.0009  0.2359

Columns 80 to 89 
 0.1226 -0.0169  0.0172 -0.0532 -0.1290  0.0558 -0.0156  0.0082 -0.0543 -0.0960

Columns 90 to 99 
-0.1444  0.0901 -0.0839 -0.0914  0.0138  0.0758 -0.0102  0.1029  0.1095

AssertionError: nn criterions don't compute the gradient w.r.t. targets - please mark these variables as volatile or not requiring gradients

Plot losses

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline

def show_plot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2) # put ticks at regular intervals
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

show_plot(plot_losses)